### Introduction

In [2]:
# Deprecated prior art with "tilted loss function": https://towardsdatascience.com/deep-quantile-regression-c85481548b5a

### Configuration

In [4]:
# Eager
import tensorflow as tf
tf.enable_eager_execution()

/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping

In [5]:
# fix random seed for reproducibility
import random
import scipy
import numpy as np
import keras

RANDOM_SEED = 0
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.set_random_seed(RANDOM_SEED)

tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.

In [6]:
# Versioning
print("SCIPY=", scipy.__version__)
print("NUMPY=", np.__version__)
print("TENSORFLOW=", tf.__version__)
print("KERAS=", keras.__version__)

SCIPY= 1.2.1
NUMPY= 1.16.2
TENSORFLOW= 1.15.0
KERAS= 2.2.5

## Credentials

In [8]:
# Get credential to read 
key=dbutils.secrets.get(scope="application-secrets", key="enterprisedatalakeprodsas")

container_list = [
  'customer',
  'customerorder',
  'foundation',
  'pricing',
  'sales',
  'cdaworkspace'
]

# Setup access to read from any container in container list
for container in container_list:
  spark.conf.set(
    "fs.azure.sas.{container}.enterprisedatalakeprod.blob.core.windows.net".format(container=container),
    key
  )

from pyspark.sql import SparkSession

# Create sparkContext
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.azure.sas.cdaworkspace.enterprisedatalakeprod.blob.core.windows.net", key)

## Read

In [10]:
import json

def view_avro(view_name, file_location):
    df = spark.read.format('avro').option("inferSchema", "true").load(file_location)
    df.createOrReplaceTempView(view_name)
    shape = (df.count(), len(df.columns))
    print("DATAFRAME_SHAPE=", shape)
    return df, shape
    
def view_orc(view_name, file_location):
    df = spark.read.format('orc').option("inferSchema", "true").load(file_location)
    df.createOrReplaceTempView(view_name) 
    shape = (df.count(), len(df.columns))
    print("DATAFRAME_SHAPE=", shape)
    return df, shape
  
def view_header_csv(view_name, file_location):
    df = spark.read.format('csv').option("inferSchema", "true").option("header", 'true').load(file_location)
    df.createOrReplaceTempView(view_name)  
    shape = (df.count(), len(df.columns))
    print("DATAFRAME_SHAPE=", shape)
    return df, shape
  
def view_blob_csv(view_name, file_name):
    file_path = 'wasbs://cdaworkspace@enterprisedatalakeprod.blob.core.windows.net/cda/prod/'
    df = spark.read.format('csv').option("inferSchema", "true").option("header", 'true').load(file_path+file_name)
    df.createOrReplaceTempView(view_name)  
    shape = (df.count(), len(df.columns))
    print("DATAFRAME_SHAPE=", shape)
    return df, shape  
  
def view_schema_csv(view_name, file_location, table_schema):
  df = spark.read.format('csv').schema(table_schema).load(file_location)
  df.createOrReplaceTempView(view_name)    
  shape = (df.count(), len(df.columns))
  print("DATAFRAME_SHAPE=", shape)
  return df, shape 

def view_file(file_name):
  file_path = 'wasbs://cdaworkspace@enterprisedatalakeprod.blob.core.windows.net/cda/prod/'
  text_rdd = sc.textFile(file_path+file_name)
  return text_rdd

In [11]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/delta/,delta/,0
dbfs:/ml/,ml/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


## Training Dataset

#### Purchase

In [14]:
purchase_regression_df, purchase_regression_shape = view_blob_csv('purchase_regression_view', 'LTV_BF_US_Regression_sales.csv') 

DATAFRAME_SHAPE= (6492696, 102)

In [15]:
purchase_regression_df.printSchema()

root
-- customer_key: decimal(9,0) (nullable = true)
-- spend_6mo_sls: double (nullable = true)
-- repeat_spend_6mo_sls: double (nullable = true)
-- item_qty_6mo_sls: string (nullable = true)
-- spend_12mo_sls: double (nullable = true)
-- repeat_spend_12mo_sls: double (nullable = true)
-- item_qty_12mo_sls: string (nullable = true)
-- spend_24mo_sls: double (nullable = true)
-- repeat_spend_24mo_sls: double (nullable = true)
-- item_qty_24mo_sls: string (nullable = true)
-- onsale_qty_6mo_sls: string (nullable = true)
-- onsale_qty_12mo_sls: string (nullable = true)
-- onsale_qty_24mo_sls: string (nullable = true)
-- num_txns_6mo_sls: string (nullable = true)
-- num_txns_12mo_sls: string (nullable = true)
-- num_txns_24mo_sls: string (nullable = true)
-- repeat_num_txns_6mo_sls: string (nullable = true)
-- repeat_num_txns_12mo_sls: string (nullable = true)
-- repeat_num_txns_24mo_sls: string (nullable = true)
-- spend_6mo_men_sls: double (nullable = true)
-- item_qty_6mo_men_sls: string (nullable = true)
-- spend_6mo_women_sls: double (nullable = true)
-- item_qty_6mo_women_sls: string (nullable = true)
-- spend_6mo_accessories_sls: double (nullable = true)
-- item_qty_6mo_accessories_sls: string (nullable = true)
-- spend_6mo_plcb_sls: double (nullable = true)
-- item_qty_6mo_plcb_sls: string (nullable = true)
-- spend_12mo_plcb_sls: double (nullable = true)
-- item_qty_12mo_plcb_sls: string (nullable = true)
-- spend_24mo_plcb_sls: double (nullable = true)
-- item_qty_24mo_plcb_sls: string (nullable = true)
-- num_plcb_txns_6mo_sls: string (nullable = true)
-- num_plcb_txns_12mo_sls: string (nullable = true)
-- num_plcb_txns_24mo_sls: string (nullable = true)
-- spend_6mo_sls_sb: double (nullable = true)
-- item_qty_6mo_sls_sb: string (nullable = true)
-- spend_12mo_sls_sb: double (nullable = true)
-- item_qty_12mo_sls_sb: string (nullable = true)
-- spend_24mo_sls_sb: double (nullable = true)
-- item_qty_24mo_sls_sb: string (nullable = true)
-- onsale_qty_6mo_sls_sb: string (nullable = true)
-- onsale_qty_12mo_sls_sb: string (nullable = true)
-- onsale_qty_24mo_sls_sb: string (nullable = true)
-- num_txns_6mo_sls_sb: string (nullable = true)
-- num_txns_12mo_sls_sb: string (nullable = true)
-- num_txns_24mo_sls_sb: string (nullable = true)
-- spend_6mo_plcb_sls_sb: double (nullable = true)
-- item_qty_plcb_6mo_sls_sb: string (nullable = true)
-- spend_12mo_plcb_sls_sb: double (nullable = true)
-- item_qty_plcb_12mo_sls_sb: string (nullable = true)
-- spend_24mo_plcb_sls_sb: double (nullable = true)
-- item_qty_plcb_24mo_sls_sb: string (nullable = true)
-- num_plcb_txns_6mo_sls_sb: string (nullable = true)
-- num_plcb_txns_12mo_sls_sb: string (nullable = true)
-- num_plcb_txns_24mo_sls_sb: string (nullable = true)
-- spend_6mo_onl_sls: double (nullable = true)
-- item_qty_6mo_onl_sls: string (nullable = true)
-- spend_12mo_onl_sls: double (nullable = true)
-- item_qty_12mo_onl_sls: string (nullable = true)
-- spend_24mo_onl_sls: double (nullable = true)
-- item_qty_24mo_onl_sls: string (nullable = true)
-- num_txns_6mo_onl_sls: string (nullable = true)
-- num_txns_12mo_onl_sls: string (nullable = true)
-- num_txns_24mo_onl_sls: string (nullable = true)
-- spend_6mo_rtl_sls: double (nullable = true)
-- item_qty_6mo_rtl_sls: string (nullable = true)
-- spend_12mo_rtl_sls: double (nullable = true)
-- item_qty_12mo_rtl_sls: string (nullable = true)
-- spend_24mo_rtl_sls: double (nullable = true)
-- item_qty_24mo_rtl_sls: string (nullable = true)
-- num_txns_6mo_rtl_sls: string (nullable = true)
-- num_txns_12mo_rtl_sls: string (nullable = true)
-- num_txns_24mo_rtl_sls: string (nullable = true)
-- stores_shopped_6mo_sls: string (nullable = true)
-- stores_shopped_12mo_sls: string (nullable = true)
-- stores_shopped_24mo_sls: string (nullable = true)
-- customer_duration_sb: string (nullable = true)
-- days_first_pur_sb: string (nullable = true)
-- days_last_pur_sb: string (nullable = true)
-- customer_duration_sb_onl: string (nu

In [16]:
display(purchase_regression_df.describe())

summary,customer_key,spend_6mo_sls,repeat_spend_6mo_sls,item_qty_6mo_sls,spend_12mo_sls,repeat_spend_12mo_sls,item_qty_12mo_sls,spend_24mo_sls,repeat_spend_24mo_sls,item_qty_24mo_sls,onsale_qty_6mo_sls,onsale_qty_12mo_sls,onsale_qty_24mo_sls,num_txns_6mo_sls,num_txns_12mo_sls,num_txns_24mo_sls,repeat_num_txns_6mo_sls,repeat_num_txns_12mo_sls,repeat_num_txns_24mo_sls,spend_6mo_men_sls,item_qty_6mo_men_sls,spend_6mo_women_sls,item_qty_6mo_women_sls,spend_6mo_accessories_sls,item_qty_6mo_accessories_sls,spend_6mo_plcb_sls,item_qty_6mo_plcb_sls,spend_12mo_plcb_sls,item_qty_12mo_plcb_sls,spend_24mo_plcb_sls,item_qty_24mo_plcb_sls,num_plcb_txns_6mo_sls,num_plcb_txns_12mo_sls,num_plcb_txns_24mo_sls,spend_6mo_sls_sb,item_qty_6mo_sls_sb,spend_12mo_sls_sb,item_qty_12mo_sls_sb,spend_24mo_sls_sb,item_qty_24mo_sls_sb,onsale_qty_6mo_sls_sb,onsale_qty_12mo_sls_sb,onsale_qty_24mo_sls_sb,num_txns_6mo_sls_sb,num_txns_12mo_sls_sb,num_txns_24mo_sls_sb,spend_6mo_plcb_sls_sb,item_qty_plcb_6mo_sls_sb,spend_12mo_plcb_sls_sb,item_qty_plcb_12mo_sls_sb,spend_24mo_plcb_sls_sb,item_qty_plcb_24mo_sls_sb,num_plcb_txns_6mo_sls_sb,num_plcb_txns_12mo_sls_sb,num_plcb_txns_24mo_sls_sb,spend_6mo_onl_sls,item_qty_6mo_onl_sls,spend_12mo_onl_sls,item_qty_12mo_onl_sls,spend_24mo_onl_sls,item_qty_24mo_onl_sls,num_txns_6mo_onl_sls,num_txns_12mo_onl_sls,num_txns_24mo_onl_sls,spend_6mo_rtl_sls,item_qty_6mo_rtl_sls,spend_12mo_rtl_sls,item_qty_12mo_rtl_sls,spend_24mo_rtl_sls,item_qty_24mo_rtl_sls,num_txns_6mo_rtl_sls,num_txns_12mo_rtl_sls,num_txns_24mo_rtl_sls,stores_shopped_6mo_sls,stores_shopped_12mo_sls,stores_shopped_24mo_sls,customer_duration_sb,days_first_pur_sb,days_last_pur_sb,customer_duration_sb_onl,days_first_pur_sb_onl,days_last_pur_sb_onl,customer_duration_sb_rtl,days_first_pur_sb_rtl,days_last_pur_sb_rtl,customer_duration,days_first_pur,days_last_pur,customer_duration_onl,days_first_pur_onl,days_last_pur_onl,customer_duration_rtl,days_first_pur_rtl,days_last_pur_rtl,days_on_books,card_status,mapped_source,net_txn_amt,validation_spend_sls,validation_item_qty_sls,validation_num_txns_sls,segment_flags
count,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696
mean,195510649.6123,47.991817372949725,2.3076863278369153,5.68607292861858,84.27506896363094,9.43248529424602,6.734093679557923,153.65171647028055,71.99389938632052,8.148729819079739,2.828251319352329,3.485142045249316,4.256150529951614,0.7188837759160469,1.2550513230588494,2.286569993225208,0.2744287394321268,0.5931203778087929,1.286569530466202,16.95563877317821,0.8896636484787442,27.421670233766875,1.7067847588472094,3.4647073850374053,0.37544239369253996,16.902894266111776,8.061920711389162,29.87642543867856,10.18429340204908,57.51325359449921,13.15848444955321,0.24957765528032708,0.44099406187643225,0.8320679959573373,137.87026244104493,16.78336294746558,255.0169460498067,25.158532577005083,502.27293904408754,41.59375663711039,7.3200567247918205,10.663294769588594,16.109683329002525,2.592796021117277,4.8146322744435786,9.412570136473066,61.821564331673436,21.897134727396473,112.66986833358406,34.33904238279827,227.75731140191596,59.08027624943179,1.1202436799338604,2.055020042207546,4.115213809804964,9.915344282253775,5.708649104251279,16.604729160276428

#### Return

In [18]:
return_regression_df, return_regression_shape = view_blob_csv('return_regression_view', 'LTV_BF_US_Regression_return.csv') 

DATAFRAME_SHAPE= (6492696, 46)

In [19]:
return_regression_df.printSchema()

root
-- customer_key: decimal(9,0) (nullable = true)
-- spend_6mo_rtn: double (nullable = true)
-- item_qty_6mo_rtn: string (nullable = true)
-- spend_12mo_rtn: double (nullable = true)
-- item_qty_12mo_rtn: string (nullable = true)
-- spend_24mo_rtn: double (nullable = true)
-- item_qty_24mo_rtn: string (nullable = true)
-- onsale_qty_6mo_rtn: string (nullable = true)
-- onsale_qty_12mo_rtn: string (nullable = true)
-- onsale_qty_24mo_rtn: string (nullable = true)
-- num_txns_6mo_rtn: string (nullable = true)
-- num_txns_12mo_rtn: string (nullable = true)
-- num_txns_24mo_rtn: string (nullable = true)
-- spend_6mo_men_rtn: double (nullable = true)
-- item_qty_6mo_men_rtn: string (nullable = true)
-- spend_6mo_women_rtn: double (nullable = true)
-- item_qty_6mo_women_rtn: string (nullable = true)
-- spend_6mo_accessories_rtn: double (nullable = true)
-- item_qty_6mo_accessories_rtn: string (nullable = true)
-- spend_6mo_rtn_sb: double (nullable = true)
-- item_qty_6mo_rtn_sb: string (nullable = true)
-- spend_12mo_rtn_sb: double (nullable = true)
-- item_qty_12mo_rtn_sb: string (nullable = true)
-- spend_24mo_rtn_sb: double (nullable = true)
-- item_qty_24mo_rtn_sb: string (nullable = true)
-- onsale_qty_6mo_rtn_sb: string (nullable = true)
-- onsale_qty_12mo_rtn_sb: string (nullable = true)
-- onsale_qty_24mo_rtn_sb: string (nullable = true)
-- num_txns_6mo_rtn_sb: string (nullable = true)
-- num_txns_12mo_rtn_sb: string (nullable = true)
-- num_txns_24mo_rtn_sb: string (nullable = true)
-- spend_6mo_plcb_rtn_sb: double (nullable = true)
-- item_qty_6mo_plcb_rtn_sb: string (nullable = true)
-- spend_12mo_plcb_rtn_sb: double (nullable = true)
-- item_qty_12mo_plcb_rtn_sb: string (nullable = true)
-- spend_24mo_plcb_rtn_sb: double (nullable = true)
-- item_qty_24mo_plcb_rtn_sb: string (nullable = true)
-- num_txns_plcb_6mo_rtn_sb: string (nullable = true)
-- num_txns_plcb_12mo_rtn_sb: string (nullable = true)
-- num_txns_plcb_24mo_rtn_sb: string (nullable = true)
-- card_status: string (nullable = true)
-- mapped_source: string (nullable = true)
-- validation_spend_rtn: double (nullable = true)
-- validation_item_qty_rtn: string (nullable = true)
-- validation_num_txns_rtn: string (nullable = true)
-- segment_flags: string (nullable = true)

In [20]:
display(return_regression_df.describe())

summary,customer_key,spend_6mo_rtn,item_qty_6mo_rtn,spend_12mo_rtn,item_qty_12mo_rtn,spend_24mo_rtn,item_qty_24mo_rtn,onsale_qty_6mo_rtn,onsale_qty_12mo_rtn,onsale_qty_24mo_rtn,num_txns_6mo_rtn,num_txns_12mo_rtn,num_txns_24mo_rtn,spend_6mo_men_rtn,item_qty_6mo_men_rtn,spend_6mo_women_rtn,item_qty_6mo_women_rtn,spend_6mo_accessories_rtn,item_qty_6mo_accessories_rtn,spend_6mo_rtn_sb,item_qty_6mo_rtn_sb,spend_12mo_rtn_sb,item_qty_12mo_rtn_sb,spend_24mo_rtn_sb,item_qty_24mo_rtn_sb,onsale_qty_6mo_rtn_sb,onsale_qty_12mo_rtn_sb,onsale_qty_24mo_rtn_sb,num_txns_6mo_rtn_sb,num_txns_12mo_rtn_sb,num_txns_24mo_rtn_sb,spend_6mo_plcb_rtn_sb,item_qty_6mo_plcb_rtn_sb,spend_12mo_plcb_rtn_sb,item_qty_12mo_plcb_rtn_sb,spend_24mo_plcb_rtn_sb,item_qty_24mo_plcb_rtn_sb,num_txns_plcb_6mo_rtn_sb,num_txns_plcb_12mo_rtn_sb,num_txns_plcb_24mo_rtn_sb,card_status,mapped_source,validation_spend_rtn,validation_item_qty_rtn,validation_num_txns_rtn,segment_flags
count,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696,6492696
mean,195510649.6123,4.708404126731888,-2.980244666635949,8.40134604177409,-3.3146505898760803,14.592674751758318,-3.6419377677312874,-1.7281758282265125,-1.9701903950797932,-2.2013091985532323,0.4576324251841163,0.8191872831580077,1.4233378857169028,1.4002834092341634,-0.319226703649935,3.1142475714257687,-1.010166416900463,0.191251189028405,-0.06889616424564826,21.448607273466962,-5.16679163441343,41.117603901355885,-6.802793772390708,81.08601634358664,-9.711233971739357,-2.1587257970702938,-2.697071530868953,-3.506062262906617,0.9901280644442326,1.8845892114059053,3.718425103642344,11.292422512928496,-6.179977747820405,21.45118816435828,-8.498250690591968,43.043136382790436,-12.663896059926362,0.5135512897487255,0.9712690332363817,1.9526981498094034,null,null,5.977790692806211,-3.7469769514862645,1.7878381389834248,null
stddev,1.0857491852506663E8,29.0342673897853,3.9007880112960986,43.00482727367231,4.909152060872182,63.83688979869292,6.04529619150927,2.5799703902112276,3.21020205939559,3.884539740444677,0.9311629935008234,1.4034250784399203,2.0715908018289415,14.193494897957443,1.2441767230907006,22.684407086232348,2.5222451935057935,2.992095396911814,0.4996231150874025,113.80013292382469,9.064576923719223,198.9017046118462,13.724753573654247,372.20710186219503,22.744664889304726,4.289009465518547,5.889909202487188,8.816305231628718,2.231288380499776,3.9434749186843607,7.4639445147257195,88.17572325170333,10.707812855558938,157.05466886387063,16.841849833889956,299.078318252441,28.658899599633777,1.868438947936844,3.3580119842372635,6.491710129636812,null,null,39.54018424155804,5.732527011675379,1.972166764043451,null
min,115,0.0,-1,0.0,-1,0.0,-1,-1,-1,-1,0,0,0,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,0.0,-1,-1,-1,-1,0,0,0,0.0,-1,0.0,-1,0.0,-1,0,0,0,NON_CARD,Card Program,0.0,-1,1,NA
max,395764197,6640.179999999973,NA,9865.57999999996,NA,14315.300000000014,NA,NA,NA,NA,NA,NA,NA,3863.7099999999873,NA,3799.33999999999,NA,662.2499999999998,NA,36946.82000000004,NA,61158.269999999844,NA,146856.58999999976,NA,NA,NA,NA,NA,NA,NA,29303.650000000074,NA,45779.29999999994,NA,72656.64000000013,NA,NA,NA,NA,NON_CARD,wifi,10245.859999999966,NA,NA,svsbsc_rtl


#### Labels

In [22]:
ground_truth_col_names = ['validation_spend_sls', 'validation_num_txns_sls', 'validation_item_qty_sls',
                    'validation_spend_rtn', 'validation_num_txns_rtn', 'validation_item_qty_rtn',
                    'net_txn_amt'] 
purchase_label_name = 'validation_spend_sls'
return_label_name = 'validation_spend_rtn'

#### Join Purchase and Return

In [24]:
to_join_cols = ['customer_key', return_label_name]
return_regression_select_df = return_regression_df.select(*to_join_cols)

In [25]:
overall_df = purchase_regression_df.join(return_regression_select_df, on='customer_key', how='inner')   

In [26]:
overall_df.printSchema()

root
-- customer_key: decimal(9,0) (nullable = true)
-- spend_6mo_sls: double (nullable = true)
-- repeat_spend_6mo_sls: double (nullable = true)
-- item_qty_6mo_sls: string (nullable = true)
-- spend_12mo_sls: double (nullable = true)
-- repeat_spend_12mo_sls: double (nullable = true)
-- item_qty_12mo_sls: string (nullable = true)
-- spend_24mo_sls: double (nullable = true)
-- repeat_spend_24mo_sls: double (nullable = true)
-- item_qty_24mo_sls: string (nullable = true)
-- onsale_qty_6mo_sls: string (nullable = true)
-- onsale_qty_12mo_sls: string (nullable = true)
-- onsale_qty_24mo_sls: string (nullable = true)
-- num_txns_6mo_sls: string (nullable = true)
-- num_txns_12mo_sls: string (nullable = true)
-- num_txns_24mo_sls: string (nullable = true)
-- repeat_num_txns_6mo_sls: string (nullable = true)
-- repeat_num_txns_12mo_sls: string (nullable = true)
-- repeat_num_txns_24mo_sls: string (nullable = true)
-- spend_6mo_men_sls: double (nullable = true)
-- item_qty_6mo_men_sls: string (nullable = true)
-- spend_6mo_women_sls: double (nullable = true)
-- item_qty_6mo_women_sls: string (nullable = true)
-- spend_6mo_accessories_sls: double (nullable = true)
-- item_qty_6mo_accessories_sls: string (nullable = true)
-- spend_6mo_plcb_sls: double (nullable = true)
-- item_qty_6mo_plcb_sls: string (nullable = true)
-- spend_12mo_plcb_sls: double (nullable = true)
-- item_qty_12mo_plcb_sls: string (nullable = true)
-- spend_24mo_plcb_sls: double (nullable = true)
-- item_qty_24mo_plcb_sls: string (nullable = true)
-- num_plcb_txns_6mo_sls: string (nullable = true)
-- num_plcb_txns_12mo_sls: string (nullable = true)
-- num_plcb_txns_24mo_sls: string (nullable = true)
-- spend_6mo_sls_sb: double (nullable = true)
-- item_qty_6mo_sls_sb: string (nullable = true)
-- spend_12mo_sls_sb: double (nullable = true)
-- item_qty_12mo_sls_sb: string (nullable = true)
-- spend_24mo_sls_sb: double (nullable = true)
-- item_qty_24mo_sls_sb: string (nullable = true)
-- onsale_qty_6mo_sls_sb: string (nullable = true)
-- onsale_qty_12mo_sls_sb: string (nullable = true)
-- onsale_qty_24mo_sls_sb: string (nullable = true)
-- num_txns_6mo_sls_sb: string (nullable = true)
-- num_txns_12mo_sls_sb: string (nullable = true)
-- num_txns_24mo_sls_sb: string (nullable = true)
-- spend_6mo_plcb_sls_sb: double (nullable = true)
-- item_qty_plcb_6mo_sls_sb: string (nullable = true)
-- spend_12mo_plcb_sls_sb: double (nullable = true)
-- item_qty_plcb_12mo_sls_sb: string (nullable = true)
-- spend_24mo_plcb_sls_sb: double (nullable = true)
-- item_qty_plcb_24mo_sls_sb: string (nullable = true)
-- num_plcb_txns_6mo_sls_sb: string (nullable = true)
-- num_plcb_txns_12mo_sls_sb: string (nullable = true)
-- num_plcb_txns_24mo_sls_sb: string (nullable = true)
-- spend_6mo_onl_sls: double (nullable = true)
-- item_qty_6mo_onl_sls: string (nullable = true)
-- spend_12mo_onl_sls: double (nullable = true)
-- item_qty_12mo_onl_sls: string (nullable = true)
-- spend_24mo_onl_sls: double (nullable = true)
-- item_qty_24mo_onl_sls: string (nullable = true)
-- num_txns_6mo_onl_sls: string (nullable = true)
-- num_txns_12mo_onl_sls: string (nullable = true)
-- num_txns_24mo_onl_sls: string (nullable = true)
-- spend_6mo_rtl_sls: double (nullable = true)
-- item_qty_6mo_rtl_sls: string (nullable = true)
-- spend_12mo_rtl_sls: double (nullable = true)
-- item_qty_12mo_rtl_sls: string (nullable = true)
-- spend_24mo_rtl_sls: double (nullable = true)
-- item_qty_24mo_rtl_sls: string (nullable = true)
-- num_txns_6mo_rtl_sls: string (nullable = true)
-- num_txns_12mo_rtl_sls: string (nullable = true)
-- num_txns_24mo_rtl_sls: string (nullable = true)
-- stores_shopped_6mo_sls: string (nullable = true)
-- stores_shopped_12mo_sls: string (nullable = true)
-- stores_shopped_24mo_sls: string (nullable = true)
-- customer_duration_sb: string (nullable = true)
-- days_first_pur_sb: string (nullable = true)
-- days_last_pur_sb: string (nullable = true)
-- customer_duration_sb_onl: string (nu

In [27]:
display(overall_df)

customer_key,spend_6mo_sls,repeat_spend_6mo_sls,item_qty_6mo_sls,spend_12mo_sls,repeat_spend_12mo_sls,item_qty_12mo_sls,spend_24mo_sls,repeat_spend_24mo_sls,item_qty_24mo_sls,onsale_qty_6mo_sls,onsale_qty_12mo_sls,onsale_qty_24mo_sls,num_txns_6mo_sls,num_txns_12mo_sls,num_txns_24mo_sls,repeat_num_txns_6mo_sls,repeat_num_txns_12mo_sls,repeat_num_txns_24mo_sls,spend_6mo_men_sls,item_qty_6mo_men_sls,spend_6mo_women_sls,item_qty_6mo_women_sls,spend_6mo_accessories_sls,item_qty_6mo_accessories_sls,spend_6mo_plcb_sls,item_qty_6mo_plcb_sls,spend_12mo_plcb_sls,item_qty_12mo_plcb_sls,spend_24mo_plcb_sls,item_qty_24mo_plcb_sls,num_plcb_txns_6mo_sls,num_plcb_txns_12mo_sls,num_plcb_txns_24mo_sls,spend_6mo_sls_sb,item_qty_6mo_sls_sb,spend_12mo_sls_sb,item_qty_12mo_sls_sb,spend_24mo_sls_sb,item_qty_24mo_sls_sb,onsale_qty_6mo_sls_sb,onsale_qty_12mo_sls_sb,onsale_qty_24mo_sls_sb,num_txns_6mo_sls_sb,num_txns_12mo_sls_sb,num_txns_24mo_sls_sb,spend_6mo_plcb_sls_sb,item_qty_plcb_6mo_sls_sb,spend_12mo_plcb_sls_sb,item_qty_plcb_12mo_sls_sb,spend_24mo_plcb_sls_sb,item_qty_plcb_24mo_sls_sb,num_plcb_txns_6mo_sls_sb,num_plcb_txns_12mo_sls_sb,num_plcb_txns_24mo_sls_sb,spend_6mo_onl_sls,item_qty_6mo_onl_sls,spend_12mo_onl_sls,item_qty_12mo_onl_sls,spend_24mo_onl_sls,item_qty_24mo_onl_sls,num_txns_6mo_onl_sls,num_txns_12mo_onl_sls,num_txns_24mo_onl_sls,spend_6mo_rtl_sls,item_qty_6mo_rtl_sls,spend_12mo_rtl_sls,item_qty_12mo_rtl_sls,spend_24mo_rtl_sls,item_qty_24mo_rtl_sls,num_txns_6mo_rtl_sls,num_txns_12mo_rtl_sls,num_txns_24mo_rtl_sls,stores_shopped_6mo_sls,stores_shopped_12mo_sls,stores_shopped_24mo_sls,customer_duration_sb,days_first_pur_sb,days_last_pur_sb,customer_duration_sb_onl,days_first_pur_sb_onl,days_last_pur_sb_onl,customer_duration_sb_rtl,days_first_pur_sb_rtl,days_last_pur_sb_rtl,customer_duration,days_first_pur,days_last_pur,customer_duration_onl,days_first_pur_onl,days_last_pur_onl,customer_duration_rtl,days_first_pur_rtl,days_last_pur_rtl,days_on_books,card_status,mapped_source,net_txn_amt,validation_spend_sls,validation_item_qty_sls,validation_num_txns_sls,segment_flags,validation_spend_rtn
8440,73.77999999999999,0.0,4,73.77999999999999,0.0,4,85.16,58.77999999999999,5,2,2,3,1,1,2,0,0,1,43.2,2,30.58,2,0.0,0,73.77999999999999,4,73.77999999999999,4,85.16,5,1,1,2,1079.6600000000003,51,1933.7000000000007,89,4890.6899999999905,191,17,37,74,8,16,41,1079.6600000000003,51,1933.7000000000007,89,4890.6899999999905,191,8,16,41,0.0,NA,0.0,NA,0.0,NA,NA,NA,NA,73.77999999999999,4,73.77999999999999,4,85.15999999999998,5,1,1,2,1,1,1,705,718,13,240,566,326,705,718,13,494,529,35,NA,NA,NA,494,529,35,529,NON_CARD,cic,0.0,0.0,NA,NA,mvmbsc_rtl,0.0
13248,0.0,0.0,NA,0.0,0.0,NA,178.93,148.94,7,NA,NA,7,0,0,3,0,0,2,0.0,0,0.0,0,0.0,0,0.0,NA,0.0,NA,0.0,NA,0,0,0,591.6700000000001,36,1162.4800000000005,68,2260.789999999998,131,26,35,57,10,18,31,0.0,NA,0.0,NA,0.0,NA,0,0,0,0.0,NA,0.0,NA,148.94,6,0,0,2,0.0,NA,0.0,NA,29.99,1,0,0,1,0,0,1,708,716,8,708,716,8,632,681,49,205,592,387,28,415,387,0,592,592,1374,NON_CARD,cic,0.0,0.0,NA,NA,mvmbmc,0.0
31156,23.39,0.0,2,23.39,0.0,2,42.38,23.39,4,1,1,2,1,1,2,0,0,1,18.9,1,0.0,0,4.49,1,23.39,2,23.39,2,23.39,2,1,1,1,188.48,13,240.96,17,279.64000000000004,20,11,12,14,7,8,9,145.57999999999998,10,145.57999999999998,10,184.26000000000002,13,4,4,5,0.0,NA,0.0,NA,0.0,NA,NA,NA,NA,23.39,2,23.39,2,42.38,4,1,1,2,1,1,2,493,494,1,160,331,171,493,494,1,321,492,171,NA,NA,NA,321,492,171,1468,NON_CARD,cic,0.0,0.0,NA,NA,mvmbsc_rtl,0.0
32912,171.78000000000003,0.0,8,267.25000000000006,0.0,13,889.0600000000003,776.1400000000003,47,5,5,28,2,5,13,1,4,12,0.0,0,163.29,7,8.49,1,171.78000000000003,8,267.25000000000006,13,889.0600000000003,47,2,5,13,1630.4699999999998,63,2756.659999999997,128,5982.219999999987,279,10,28,78,18,38,86,1480.0100000000007,60,2544.599999999998,120,5746.189999999986,267,16,35,82,125.44000000000001,6,125.44000000000001,6,599.2400000000001,25,1,1,5,46.339999999999996,2,141.80999999999997,7,289.8200000000001,22,1,4,8,1,1,1,701,709,8,589,630,4

## Model Training

In [29]:
import warnings
warnings.filterwarnings('ignore')

# General libraries
import sys,os,time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sb
from statistics import median 

# ML libraries 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor as rfr
from keras import layers
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from keras import regularizers
from keras.layers import Dropout
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from keras.constraints import max_norm
from keras.models import model_from_json
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import MinMaxScaler
import keras.backend as K

In [30]:
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F


CAST_COLUMN_TAG = '_'
  
def na_and_fill(df):
  return df \
        .dropna() 
         # TODO: revisit this assumption vs fillna(0)?
  
def cast_column(df):  
  for col_name in df_in.columns:
    cast_col_name = col_name + CAST_COLUMN_TAG
    df = df \
          .withColumn(cast_col_name, df[col_name].cast(DoubleType())) \
          .drop(col_name)
  return df

def drop_null(df): 
  null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]) \
                      .collect()[0] \
                      .asDict()
  to_drop = [k for k, v in null_counts.items() if v > 0]
  df = df.drop(*to_drop)
  
  return df, to_drop

# TODO: Cap outliers?

def filter_negative(df):
  for col_name in ground_truth_col_names:
    try:
      df = df.filter(df[col_name] > 0)
    except:
      pass
  return df


In [31]:
df_in = overall_df.drop('customer_key')

df = na_and_fill(df_in)
df = cast_column(df)
df = filter_negative(df)
df_final, to_drop = drop_null(df)

print("COLUMN_TO_DROP=", to_drop)

assert df_final.count() > 0, "DATAFRAME_LENGTH_ERROR"  

COLUMN_TO_DROP= ['item_qty_6mo_sls_', 'item_qty_12mo_sls_', 'item_qty_24mo_sls_', 'onsale_qty_6mo_sls_', 'onsale_qty_12mo_sls_', 'onsale_qty_24mo_sls_', 'num_txns_6mo_sls_', 'num_txns_12mo_sls_', 'num_txns_24mo_sls_', 'repeat_num_txns_6mo_sls_', 'repeat_num_txns_12mo_sls_', 'repeat_num_txns_24mo_sls_', 'item_qty_6mo_men_sls_', 'item_qty_6mo_women_sls_', 'item_qty_6mo_accessories_sls_', 'item_qty_6mo_plcb_sls_', 'item_qty_12mo_plcb_sls_', 'item_qty_24mo_plcb_sls_', 'num_plcb_txns_6mo_sls_', 'num_plcb_txns_12mo_sls_', 'num_plcb_txns_24mo_sls_', 'item_qty_6mo_sls_sb_', 'item_qty_12mo_sls_sb_', 'item_qty_24mo_sls_sb_', 'onsale_qty_6mo_sls_sb_', 'onsale_qty_12mo_sls_sb_', 'onsale_qty_24mo_sls_sb_', 'num_txns_6mo_sls_sb_', 'num_txns_12mo_sls_sb_', 'num_txns_24mo_sls_sb_', 'item_qty_plcb_6mo_sls_sb_', 'item_qty_plcb_12mo_sls_sb_', 'item_qty_plcb_24mo_sls_sb_', 'num_plcb_txns_6mo_sls_sb_', 'num_plcb_txns_12mo_sls_sb_', 'num_plcb_txns_24mo_sls_sb_', 'item_qty_6mo_onl_sls_', 'item_qty_12mo_onl_sls_', 'item_qty_24mo_onl_sls_', 'num_txns_6mo_onl_sls_', 'num_txns_12mo_onl_sls_', 'num_txns_24mo_onl_sls_', 'item_qty_6mo_rtl_sls_', 'item_qty_12mo_rtl_sls_', 'item_qty_24mo_rtl_sls_', 'num_txns_6mo_rtl_sls_', 'num_txns_12mo_rtl_sls_', 'num_txns_24mo_rtl_sls_', 'stores_shopped_6mo_sls_', 'stores_shopped_12mo_sls_', 'stores_shopped_24mo_sls_', 'customer_duration_sb_', 'days_first_pur_sb_', 'days_last_pur_sb_', 'customer_duration_sb_onl_', 'days_first_pur_sb_onl_', 'days_last_pur_sb_onl_', 'customer_duration_sb_rtl_', 'days_first_pur_sb_rtl_', 'days_last_pur_sb_rtl_', 'customer_duration_', 'days_first_pur_', 'days_last_pur_', 'customer_duration_onl_', 'days_first_pur_onl_', 'days_last_pur_onl_', 'customer_duration_rtl_', 'days_first_pur_rtl_', 'days_last_pur_rtl_', 'days_on_books_', 'card_status_', 'mapped_source_', 'validation_item_qty_sls_', 'validation_num_txns_sls_', 'segment_flags_']

In [32]:
train_df, test_df, val_df = df_final.randomSplit([.7, .25, .05])
train_df.cache()
test_df.cache()
val_df.cache()

train_df.count(), test_df.count(), val_df.count() 

Out[21]: (4544001, 1623432, 325263)

In [33]:
display(train_df)

spend_6mo_sls_,repeat_spend_6mo_sls_,spend_12mo_sls_,repeat_spend_12mo_sls_,spend_24mo_sls_,repeat_spend_24mo_sls_,spend_6mo_men_sls_,spend_6mo_women_sls_,spend_6mo_accessories_sls_,spend_6mo_plcb_sls_,spend_12mo_plcb_sls_,spend_24mo_plcb_sls_,spend_6mo_sls_sb_,spend_12mo_sls_sb_,spend_24mo_sls_sb_,spend_6mo_plcb_sls_sb_,spend_12mo_plcb_sls_sb_,spend_24mo_plcb_sls_sb_,spend_6mo_onl_sls_,spend_12mo_onl_sls_,spend_24mo_onl_sls_,spend_6mo_rtl_sls_,spend_12mo_rtl_sls_,spend_24mo_rtl_sls_,net_txn_amt_,validation_spend_sls_,validation_spend_rtn_
0.0,-16.450000000000017,0.0,-16.450000000000017,0.0,-16.450000000000017,0.0,0.0,0.0,0.0,0.0,0.0,280.5100000000001,375.80000000000007,375.8000000000002,264.06000000000006,359.35000000000014,359.3500000000001,0.0,0.0,0.0,0.0,0.0,0.0,3.5999999999999943,3.5999999999999943,0.0
0.0,0.0,0.0,0.0,0.0,-47.19,0.0,0.0,0.0,0.0,0.0,0.0,165.38,616.5000000000001,980.5700000000003,159.89,541.0600000000001,905.1300000000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,-6.439999999999998,0.0,0.0,0.0,0.0,0.0,0.0,15.349999999999994,34.129999999999995,60.76000000000005,15.349999999999994,34.129999999999995,60.76000000000005,0.0,0.0,0.0,0.0,0.0,0.0,107.45000000000002,107.45000000000002,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Ground truth columns renamed

In [35]:
ground_truth_col_renamed = [x+CAST_COLUMN_TAG for x in ground_truth_col_names]
purchase_label_renamed = purchase_label_name + CAST_COLUMN_TAG
return_label_renamed = return_label_name + CAST_COLUMN_TAG

#### Input pipeline

In [37]:
# https://dwgeek.com/python-pyspark-iterator-how-to-create-and-use.html/
def get_dict_iter(df):
  return df.rdd.map(lambda r: r.asDict()).toLocalIterator()

In [38]:
# test
sample_iter = get_dict_iter(val_df)

In [39]:
def get_df_batch(df_iter, batch_size=100):
  batch = []
  for i in range(batch_size):
    try:
      next_is = next(df_iter)
      batch.append(next_is)
      # print(next_is)
    except:
      pass
  return batch

def get_tf_dataset_batch(df_iter, batch_size):
  batch_dict = get_df_batch(df_iter, batch_size)
  batch_pd = pd.DataFrame.from_dict(batch_dict) 
  
  purchase_label_col = batch_pd.pop(purchase_label_renamed)
  return_label_col = batch_pd.pop(return_label_renamed)
  net_label_col = purchase_label_col - return_label_col
  dict_labels = (dict(batch_pd), net_label_col)
  
  tf_dataset = tf.data.Dataset.from_tensor_slices(dict_labels)
  return tf_dataset.batch(batch_size), batch_pd

In [40]:
# test
sample_ds, sample_pd = get_tf_dataset_batch(sample_iter, batch_size=1)

for feature_batch, label_batch in sample_ds.take(1):
  print('EVERY_FEATURE=', list(feature_batch.keys()))
  print('FEATURE_BATH repeat_spend_12mo_sls_=', feature_batch['repeat_spend_12mo_sls_'])
  print('TARGET_BATCH=', label_batch )

EVERY_FEATURE= ['net_txn_amt_', 'repeat_spend_12mo_sls_', 'repeat_spend_24mo_sls_', 'repeat_spend_6mo_sls_', 'spend_12mo_onl_sls_', 'spend_12mo_plcb_sls_', 'spend_12mo_plcb_sls_sb_', 'spend_12mo_rtl_sls_', 'spend_12mo_sls_', 'spend_12mo_sls_sb_', 'spend_24mo_onl_sls_', 'spend_24mo_plcb_sls_', 'spend_24mo_plcb_sls_sb_', 'spend_24mo_rtl_sls_', 'spend_24mo_sls_', 'spend_24mo_sls_sb_', 'spend_6mo_accessories_sls_', 'spend_6mo_men_sls_', 'spend_6mo_onl_sls_', 'spend_6mo_plcb_sls_', 'spend_6mo_plcb_sls_sb_', 'spend_6mo_rtl_sls_', 'spend_6mo_sls_', 'spend_6mo_sls_sb_', 'spend_6mo_women_sls_']
FEATURE_BATH repeat_spend_12mo_sls_= tf.Tensor([0.], shape=(1,), dtype=float64)
TARGET_BATCH= tf.Tensor([0.], shape=(1,), dtype=float64)

In [41]:
# test
from tensorflow import feature_column

sample_batch = next(iter(sample_ds))[0]
col = feature_column.numeric_column('repeat_spend_12mo_sls_')
sample_layer = tf.keras.layers.DenseFeatures(col)
layer_numpy = sample_layer(sample_batch).numpy()

print("layer_numpy=", layer_numpy)
assert len(layer_numpy)>0, "BATCHING_FAILED"

layer_numpy= [[0.]]

In [42]:
from tensorflow.keras.layers import Layer

def avoid_snooping_truth(xy_pd):
  x_pd = xy_pd
  
  for col in ground_truth_col_renamed:
    try:
      x_pd.drop(col)
    except:
      pass

  return x_pd


def get_numeric_features(x_pd):
  feature_columns = []
  numeric_columns = x_pd.select_dtypes(include=['float64']).columns
  for header in numeric_columns:
    feature_columns.append(feature_column.numeric_column(header))
  return feature_columns

  
def get_features(xy_pd):
  x_pd = avoid_snooping_truth(xy_pd) 

  feature_columns = []
  feature_columns += get_numeric_features(x_pd)
  # feature_columns += get_indicator_features(x_pd)

  feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
  # print("feature_columns", feature_columns)
  
  return feature_layer


def get_indicator_features(x_pd):
  feature_columns = []
  indicator_columns = x_pd.select_dtypes(include=['object']).columns
  
  for col in indicator_columns:
    category_feature = feature_column.categorical_column_with_vocabulary_list(col, indicator_columns)
    onehot_feature = feature_column.indicator_column(category_feature)
    feature_columns.append(onehot_feature)
  return feature_columns


In [43]:
# test
sample_layer = get_features(sample_pd)

sample_layer

Out[30]: <tensorflow.python.feature_column.dense_features.DenseFeatures at 0x7efd0e1185f8>

In [44]:
def neural_net_model(input_layer, 
                     hidden_layer_neuron_count=256, num_hidden_layers=4):
    layers = [input_layer]
    for i in range(num_hidden_layers):
      layers.append(tf.keras.layers.Dense(hidden_layer_neuron_count, kernel_initializer='normal', activation='relu'))
    layers.append(tf.keras.layers.Dense(1, kernel_initializer='normal', activation='linear'))
    model = tf.keras.Sequential(layers)
    return model

In [45]:
from collections import defaultdict

from keras.callbacks import History 
from keras.callbacks import EarlyStopping


def train_model(model_key, 
                loss_function,
                train_df, test_df, 
                learning_rate,
                layer_neurons_nonlinear, num_layers_nonlinear,
                whole_epochs=1, epoch_per_batch=1000 # relies on EarlyStopping
               ):

  num_batches = 10
  batch_size = int(train_df.count()/num_batches)

  test_iter = get_dict_iter(test_df)
  test_ds, test_pd = get_tf_dataset_batch(test_iter, batch_size)
  feature_layer = get_features(test_pd)
  
  training_model = neural_net_model(feature_layer, layer_neurons_nonlinear, num_layers_nonlinear)
  # keras_wrapper = keras.wrappers.scikit_learn.KerasRegressor(training_model)
  training_model.compile(loss=loss_function, 
                optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate), 
                metrics=[loss_function, 'mean_absolute_error'])

  training_history = []
      
  while whole_epochs > 0:
    train_iter = get_dict_iter(train_df) # .orderBy(rand()
    
    while num_batches > 0: 
      print("\n\n*******************\n", 
            "MODEL_KEY=", model_key, 
            "REMAINING_EPOCHS=", whole_epochs,
            "REMAINING_BATCHES=", num_batches,  
            "\n*******************\n")

      train_ds, train_pd = get_tf_dataset_batch(train_iter, batch_size)    

      history = History()
      training_model.fit(train_ds,
                validation_data=test_ds,  
                shuffle=True,         
                epochs=epoch_per_batch,
                callbacks=[history,
                           EarlyStopping(monitor='val_loss', mode='auto', patience=5)])  # min_delta=1
      training_history.append(history)

      num_batches-=1
    whole_epochs-=1
    
  return training_model, training_history

In [46]:
def train_models(hyper_params):
  param_model = {}
  param_history = {}

  for loss in hyper_params['loss_function']:
    for height in hyper_params['layer_neurons_nonlinear']:
      for depth in hyper_params['num_layers_nonlinear']:
        for rate in hyper_params['learning_rate']:
          for whole_epochs in hyper_params['whole_dataset_epochs']:
            for epoch_per_batch in hyper_params['epoch_per_batch']:
              try:
                model_key = '-' + 'depth=' + str(depth) + '-' + 'height=' + str(height) +  \
                  '-' + 'whole_epochs=' + str(whole_epochs) + '-' + 'epoch_per_batch=' + str(epoch_per_batch) + \
                  '-' + 'loss=' + loss + '-' + 'rate=' + str(rate)
                model, history = train_model(model_key, 
                                             loss,
                                             train_df, test_df, 
                                             rate,
                                             layer_neurons_nonlinear=height, num_layers_nonlinear=depth,
                                             whole_epochs=whole_epochs, epoch_per_batch=epoch_per_batch)
                param_model[model_key] = model
                param_history[model_key] = history
              except Exception as e:
                print("TRAINING_EXCEPTION", model_key, e)

  return param_model, param_history 

In [47]:
def choose_model(param_model, param_history):
  COMPARISON_BATCH_SIZE = 250000
  test_iter = get_dict_iter(test_df)
  test_ds, test_pd = get_tf_dataset_batch(test_iter, batch_size=COMPARISON_BATCH_SIZE)

  chosen_key = None
  chosen_model = None
  chosen_test_loss = None
  chosen_test_error = None
  for model_key, model in param_model.items():
    try:
      test_evaluation = model.evaluate(test_ds)
      test_loss = test_evaluation[0] 
      test_objective_error = test_evaluation[1]
      print("MODEL_EVALUATED=", model_key, "TRAIN_LOSS=", test_loss, "TRAIN_ERROR=", test_objective_error)
      if chosen_model == None or (test_loss < chosen_test_loss):
        chosen_key = model_key
        chosen_model = model
        chosen_test_loss = test_loss
        chosen_test_error = test_objective_error
        print("MODEL_SELECTED=", model_key)
    except Exception as e:
      print(model_key, "FAILED_SELECTION", e)
      pass
  return chosen_key, chosen_model, chosen_test_loss, chosen_test_error

In [48]:
def measure_model_performance(chosen_key, chosen_model):
  try:
    MEASUREMENT_BATCH_SIZE = 50000
    val_iter = get_dict_iter(val_df)
    val_ds, val_pd = get_tf_dataset_batch(val_iter, batch_size=MEASUREMENT_BATCH_SIZE)

    val_evaluation = chosen_model.evaluate(val_ds)
    loss = val_evaluation[0]
    error = val_evaluation[1]
    print("CHOSEN_MODEL=", chosen_key, "VALIDATION_ERROR=", error)
  except Exception as e:
    print(e)
    pass

### Run Model Training and Selection

#### Loss = mean_absolute_percentage_error

In [51]:
hyper_params = {}
hyper_params['loss_function'] = ['mean_absolute_percentage_error'] # 'mean_squared_error', mean_absolute_error, mean_squared_logarithmic_error, cosine_similarity, huber_loss, log_cosh
hyper_params['num_layers_nonlinear'] = [1, 2, 3] 
hyper_params['layer_neurons_nonlinear'] = [6, 12, 18] 
hyper_params['learning_rate'] = [10**-12]    

hyper_params['whole_dataset_epochs'] = [1] # 10
hyper_params['epoch_per_batch'] = [5]


In [52]:
param_model, param_history = train_models(hyper_params)

*******************
 MODEL_KEY= -depth=1-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 REMAINING_EPOCHS= 1 REMAINING_BATCHES= 10 
*******************

Epoch 1/5
 1/Unknown - 18s 18s/step - loss: 1945880192.0000 - mean_absolute_percentage_error: 1945880192.0000 - mean_absolute_error: 49.38411/1 [==============================] - 35s 35s/step - loss: 1945880192.0000 - mean_absolute_percentage_error: 1945880192.0000 - mean_absolute_error: 49.3841 - val_loss: 0.0000e+00 - val_mean_absolute_percentage_error: 0.0000e+00 - val_mean_absolute_error: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 32s 32s/step - loss: 262421216.0000 - mean_absolute_percentage_error: 262421216.0000 - mean_absolute_error: 46.2176 - val_loss: 330273888.0000 - val_mean_absolute_percentage_error: 330273888.0000 - val_mean_absolute_error: 45.5441
Epoch 3/5
1/1 [==============================] - 31s 31s/step - loss: 328622400.0000 - mean_absolute_percentage_error: 328622400.0000 - mean_absolute_error: 46.1776 - val_loss: 983044352.0000 - val_mean_absolute_percentage_error: 983044352.0000 - val_mean_absolute_error: 47.1539
Epoch 4/5
1/1 [==============================] - 31s 31s/step - loss: 976272640.0000 - mean_absolute_percentage_error: 976272640.0000 - mean_absolute_error: 47.7854 - val_loss: 408028768.0000 - val_mean_absolute_percentage_error: 408028768.0000 - val_mean_absolute_error: 45.6055
Epoch 5/5
1/1 [==============================] - 31s 31s/step - loss: 405716352.0000 - mean_absolute_percentage_error: 405716352.0000 - mean_absolute_error: 46.2382 - val_loss: 742395520.0000 - val_mean_absolute_percentage_error: 742395520.0000 - val_mean_absolute_error: 46.7869


*******************
 MODEL_KEY= -depth=1-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 REMAINING_EPOCHS= 1 REMAINING_BATCHES= 9 
*******************

Epoch 1/5
 1/Unknown - 15s 15s/step - loss: 739852032.0000 - mean_absolute_percentage_error: 739852032.0000 - mean_absolute_error: 47.37471/1 [==============================] - 30s 30s/step - loss: 739852032.0000 - mean_absolute_percentage_error: 739852032.0000 - mean_absolute_error: 47.3747 - val_loss: 0.0000e+00 - val_mean_absolute_percentage_error: 0.0000e+00 - val_mean_absolute_error: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 31s 31s/step - loss: 316566400.0000 - mean_absolute_percentage_error: 316566400.0000 - mean_absolute_error: 46.2170 - val_loss: 540091712.0000 - val_mean_absolute_percentage_error: 540091712.0000 - val_mean_absolute_error: 46.4710
Epoch 3/5
1/1 [==============================] - 30s 30s/step - loss: 538442432.0000 - mean_absolute_percentage_error: 538442432.0000 - mean_absolute_error: 47.0589 - val_loss: 276191232.0000 - val_mean_absolute_percentage_error: 276191232.0000 - val_mean_absolute_error: 45.6542
Epoch 4/5
1/1 [==============================] - 30s 30s/step - loss: 275568544.0000 - mean_absolute_percentage_error: 275568544.0000 - mean_absolute_error: 46.2383 - val_loss: 403233568.0000 - val_mean_absolute_percentage_error: 403233568.0000 - val_mean_absolute_error: 46.2621
Epoch 5/5
1/1 [==============================] - 30s 30s/step - loss: 402118848.0000 - mean_absolute_percentage_error: 402118848.0000 - mean_absolute_error: 46.8500 - val_loss: 235490400.0000 - val_mean_absolute_percentage_error: 235490400.0000 - val_mean_absolute_error: 45.6679


*******************
 MODEL_KEY= -depth=1-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 REMAINING_EPOCHS= 1 REMAINING_BATCHES= 8 
*******************

Epoch 1/5
 1/Unknown - 15s 15s/step - loss: 235898224.0000 - mean_absolute_percentage

In [53]:
chosen_key, chosen_model, chosen_test_loss, chosen_test_error = choose_model(param_model, param_history)

1/1 [==============================] - 32s 32s/step - loss: 8739077.0000 - mean_absolute_percentage_error: 8739077.0000 - mean_absolute_error: 45.5943
MODEL_EVALUATED= -depth=1-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 8739077.0 TRAIN_ERROR= 8739077.0
MODEL_SELECTED= -depth=1-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12
1/1 [==============================] - 9s 9s/step - loss: 8143327.0000 - mean_absolute_percentage_error: 8143327.0000 - mean_absolute_error: 45.6253
MODEL_EVALUATED= -depth=2-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 8143327.0 TRAIN_ERROR= 8143327.0
MODEL_SELECTED= -depth=2-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12
1/1 [==============================] - 8s 8s/step - loss: 3672998.5000 - mean_absolute_percentage_error: 3672998.5000 - mean_absolute_error: 45.6250
MODEL_EVALUATED= -depth=3-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 3672998.5 TRAIN_ERROR= 3672998.5
MODEL_SELECTED= -depth=3-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12
1/1 [==============================] - 9s 9s/step - loss: 72781640.0000 - mean_absolute_percentage_error: 72781640.0000 - mean_absolute_error: 45.7747
MODEL_EVALUATED= -depth=1-height=12-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 72781640.0 TRAIN_ERROR= 72781640.0
1/1 [==============================] - 8s 8s/step - loss: 10194882.0000 - mean_absolute_percentage_error: 10194882.0000 - mean_absolute_error: 45.6218
MODEL_EVALUATED= -depth=2-height=12-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 10194882.0 TRAIN_ERROR= 10194882.0
1/1 [==============================] - 9s 9s/step - loss: 6754391.0000 - mean_absolute_percentage_error: 6754391.0000 - mean_absolute_error: 45.6325
MODEL_EVALUATED= -depth=3-height=12-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 6754391.0 TRAIN_ERROR= 6754391.0
1/1 [==============================] - 9s 9s/step - loss: 132566992.0000 - mean_absolute_percentage_error: 132566992.0000 - mean_absolute_error: 45.5887
MODEL_EVALUATED= -depth=1-height=18-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 132566992.0 TRAIN_ERROR= 132566990.0
1/1 [==============================] - 9s 9s/step - loss: 42452596.0000 - mean_absolute_percentage_error: 42452596.0000 - mean_absolute_error: 45.6428
MODEL_EVALUATED= -depth=2-height=18-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 42452596.0 TRAIN_ERROR= 42452596.0
1/1 [==============================] - 9s 9s/step - loss: 11508606.0000 - mean_absolute_percentage_error: 11508606.0000 - mean_absolute_error: 45.6334
MODEL_EVALUATED= -depth=3-height=18-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 TRAIN_LOSS= 11508606.0 TRAIN_ERROR= 11508606.0

In [54]:
measure_model_performance(chosen_key, chosen_model)

1/1 [==============================] - 2s 2s/step - loss: 3666583.2500 - mean_absolute_percentage_error: 3666583.2500 - mean_absolute_error: 46.6171
CHOSEN_MODEL= -depth=3-height=6-whole_epochs=1-epoch_per_batch=5-loss=mean_absolute_percentage_error-rate=1e-12 VALIDATION_ERROR= 3666583.2

### Plot

#### Plot Helper Classes

In [57]:
from collections import defaultdict


class model_history():
  
  def __init__(self):
    self.loss_history = defaultdict(list)
    self.error_history = defaultdict(list)
    self.percentage_error_history = defaultdict(list)
  
  def add(self, key, values):
    if key in ["loss", "val_loss"]:
      self.loss_history[key].extend(values)
    if key in ["mean_absolute_error", "val_mean_absolute_error"]:
      self.error_history[key].extend(values)
    if key in ["mean_absolute_percentage_error", "val_mean_absolute_percentage_error"]:
      self.percentage_error_history[key].extend(values)
      
# test
h = model_history()
h.add('loss', [1,2,3])
h.add('loss', [3,2,1])
h.loss_history

Out[40]: defaultdict(list, {'loss': [1, 2, 3, 3, 2, 1]})

In [58]:
model_train_history = {}

for model_key in param_history.keys():
  try:
    history_arr = param_history[model_key]
    for batch_id in range(len(history_arr)):
      history_batch = history_arr[batch_id]
      # print(history_batch.history.keys())
      for metric_name, metric_history in history_batch.history.items():
        if model_key not in model_train_history:
          this_history = model_history()
        else:
          this_history = model_train_history[model_key]
        this_history.add(metric_name, metric_history)
        model_train_history[model_key] = this_history 
  except:
    pass

In [59]:
def plot_learning_curves(model_key, train_history):
  try:
    print("model_key=", model_key)
    pd.DataFrame(train_history).plot(figsize=(8, 5), title=model_key)
    plt.grid(True)
    # plt.gca().set_ylim(0, 1)
    # plt.show()
  except:
    pass

#### Plot the Selected Model

In [61]:
plot_learning_curves(chosen_key, model_train_history[chosen_key].loss_history)

In [62]:
plot_learning_curves(chosen_key, model_train_history[chosen_key].percentage_error_history)

In [63]:
  plot_learning_curves(chosen_key, model_train_history[chosen_key].error_history)

#### Plot All Models

In [65]:
for key in model_train_history.keys():
  plot_learning_curves(key, model_train_history[key].loss_history)

In [66]:
for key in model_train_history.keys():
  plot_learning_curves(key, model_train_history[key].error_history)

In [67]:
for key in model_train_history.keys():
  plot_learning_curves(key, model_train_history[key].percentage_error_history)